In [1]:
# importing libraries
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense,
                                     Flatten,
                                     Dropout,
                                     BatchNormalization,
                                     Conv2D,
                                     MaxPooling2D,)
from tensorflow.keras.regularizers import l2

In [2]:
#importing data
def load_data_from_directories(view1_dir, view2_dir, labels_dir):
    # List all files in each directory
    files_view1 = os.listdir(view1_dir)
    files_view2 = os.listdir(view2_dir)
    files_label = os.listdir(labels_dir)
    # Initialize empty lists to store data from each view and labels
    view1_data = []
    view2_data = []
    labels_data = []
    # Iterate through the files in the directory
    for filename in files_view1:
        if filename.endswith('_samples_view1.npy'):
            # Extract the common "number" part of the file name
            common_number = filename.split('_')[0]
            # Check if corresponding files exist for view2 and labels
            if common_number + '_samples_view2.npy' in files_view2 and common_number + '_labels.npy' in files_label:
                # Load data from the NumPy files
                data_view1 = np.load(os.path.join(view1_dir, filename))
                data_view2 = np.load(os.path.join(view2_dir, common_number + '_samples_view2.npy'))
                data_labels = np.load(os.path.join(labels_dir, common_number + '_labels.npy'))
                # Append data to respective lists
                view1_data.append(data_view1)
                view2_data.append(data_view2)
                labels_data.append(data_labels)
    view1_data = np.array(view1_data)
    view2_data = np.array(view2_data)
    labels_data = np.array(labels_data)

    return view1_data, view2_data, labels_data
view1_dir = (r"C:\Users\user\Desktop\data\view1")
view2_dir = (r"C:\Users\user\Desktop\data\view2")
labels_dir = (r"C:\Users\user\Desktop\data\labels")

view1_data, view2_data, labels_data = load_data_from_directories(view1_dir, view2_dir, labels_dir)

print(" shape view 1 data: ", view1_data.shape)
print(" shape view 2 data: ", view2_data.shape)
print(" shape labels data: ", labels_data.shape)

 shape view 1 data:  (13683, 32, 32)
 shape view 2 data:  (13683, 32, 32)
 shape labels data:  (13683, 1)


In [3]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [4]:
"""
    Split the dataset into labeled, unlabeled, and test sets.

    Parameters:
    - dataset (list or array-like): The input dataset to be split.
    - labeled_size (int): The target size for the labeled set (default: 130).
    - test_size (float): The proportion of the dataset to include in the test split (default: 0.2).
    - random_seed (int): Seed for reproducibility (default: None).

    Returns:
    - labeled_set_view1: Subset of the dataset with labeled data (approximately 100-130 points).
    - labeled_set_view2: Subset of the dataset with labeled data (approximately 100-130 points).
    - label_labeled_set: Labels corresponding to the labeled data points.
    - unlabeled_set: Subset of the dataset with unlabeled data.
    - test_set: Subset of the dataset for testing.
    - label_test_set: Labels corresponding to the test data points.
    """
def split_dataset(view1_data, view2_data, labels_data, labeled_size=120, test_size=0.2, random_seed=42):
    # Split view1 data into labeled and test sets
    labeled_set_view1, test_set_view1, label_labeled_set_view1, label_test_set_view1 = train_test_split(
        view1_data, labels_data, train_size=labeled_size, test_size=test_size, random_state=random_seed)
    
    # Split view2 data into labeled and test sets
    labeled_set_view2, test_set_view2, label_labeled_set_view2, label_test_set_view2 = train_test_split(
        view2_data, labels_data, train_size=labeled_size, test_size=test_size, random_state=random_seed)
    
    return labeled_set_view1, test_set_view1, label_labeled_set_view1, label_test_set_view1, labeled_set_view2, test_set_view2, label_labeled_set_view2, label_test_set_view2


In [5]:
labeled_set_view1, test_set_view1, label_labeled_set_view1, label_test_set_view1, labeled_set_view2, test_set_view2, label_labeled_set_view2, label_test_set_view2 = split_dataset(view1_data, view2_data, labels_data)

print("Shape of labeled_set_view1:", labeled_set_view1.shape)
print("Shape of test_set_view1:", test_set_view1.shape)
print("Shape of labeled_set_view2:", labeled_set_view2.shape)
print("Shape of test_set_view2:", test_set_view2.shape)
print("Shape of label_labeled_set_view1:", label_labeled_set_view1.shape)
print("Shape of label_test_set_view1:", label_test_set_view1.shape)
print("Shape of label_labeled_set_view2:", label_labeled_set_view2.shape)
print("Shape of label_test_set_view2:", label_test_set_view2.shape)


Shape of labeled_set_view1: (120, 32, 32)
Shape of test_set_view1: (2737, 32, 32)
Shape of labeled_set_view2: (120, 32, 32)
Shape of test_set_view2: (2737, 32, 32)
Shape of label_labeled_set_view1: (120, 1)
Shape of label_test_set_view1: (2737, 1)
Shape of label_labeled_set_view2: (120, 1)
Shape of label_test_set_view2: (2737, 1)


In [6]:
excluded_indices = np.concatenate((labeled_set_view1, test_set_view1)).astype(int)
unlabeled_set_view1 = np.delete(view1_data, excluded_indices, axis=0).astype(int)

In [7]:
excluded_indices = np.concatenate((labeled_set_view2, test_set_view2)).astype(int)
unlabeled_set_view2 = np.delete(view2_data, excluded_indices, axis=0).astype(int)

In [8]:
print("Shape of unlabeled_set_view1:", unlabeled_set_view1.shape)

Shape of unlabeled_set_view1: (13679, 32, 32)


In [9]:
print("Shape of unlabeled_set_view2:", unlabeled_set_view2.shape)

Shape of unlabeled_set_view2: (13680, 32, 32)


In [10]:
# Number of classes 
num_classes = len(np.unique(labels_data))
print(num_classes)

6


In [11]:
# Sequential model
cnn_classifier = Sequential()

#1. Input Layer: BatchNormalization with input shape (32, 32, 1).
cnn_classifier.add(BatchNormalization(input_shape=(32, 32, 1)))

#2. Convolutional Layer 1: 32 filters, each with a 3x3 kernel and ReLU activation.
cnn_classifier.add(Conv2D(32, (3, 3), activation='relu'))

#3. Max Pooling Layer 1: 2x2 pooling with a stride of 2.
cnn_classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))

#4. Convolutional Layer 2: 32 filters, each with a 3x3 kernel and ReLU activation.
cnn_classifier.add(Conv2D(32, (3, 3), activation='relu'))

#5. Max Pooling Layer 2: 2x2 pooling with a stride of 2.
cnn_classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))

#6. Convolutional Layer 3: 32 filters, each with a 3x3 kernel and ReLU activation.
cnn_classifier.add(Conv2D(32, (3, 3), activation='relu'))

#7. Max Pooling Layer 3: 2x2 pooling with a stride of 2.
cnn_classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))

#8. BatchNormalization layer
cnn_classifier.add(BatchNormalization())

#9. Flatten
cnn_classifier.add(Flatten())

#10. Dropout layer-Dropout=0.1
cnn_classifier.add(Dropout(0.1))

#11. Fully Connected Layer 1: 16 neurons, ReLU activation, and L2 regularization with a weight decay of 0.001.
cnn_classifier.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)))

#12. Dropout Layer with a dropout rate of 0.1.
cnn_classifier.add(Dropout(0.1))

#13. Fully Connected Layer 2: 16 neurons, ReLU activation, and L2 regularization with a weight decay of 0.001.
cnn_classifier.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)))

#14. Dropout layer-Dropout=0.1
cnn_classifier.add(Dropout(0.1))

#15. Output Layer: Dense layer with the number of neurons equal to the number of classes and softmax activation.
cnn_classifier.add(Dense(6, activation='softmax'))

cnn_classifier.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)

Epoch 1/38
4/4 [==============================] - 4s 36ms/step - loss: 2.1797 - accuracy: 0.1250
Epoch 2/38
4/4 [==============================] - 0s 37ms/step - loss: 1.8368 - accuracy: 0.1833
Epoch 3/38
4/4 [==============================] - 0s 35ms/step - loss: 1.7236 - accuracy: 0.2167
Epoch 4/38
4/4 [==============================] - 0s 33ms/step - loss: 1.6109 - accuracy: 0.3250
Epoch 5/38
4/4 [==============================] - 0s 33ms/step - loss: 1.5545 - accuracy: 0.3833
Epoch 6/38
4/4 [==============================] - 0s 33ms/step - loss: 1.4861 - accuracy: 0.4250
Epoch 7/38
4/4 [==============================] - 0s 43ms/step - loss: 1.4031 - accuracy: 0.5167
Epoch 8/38
4/4 [==============================] - 0s 39ms/step - loss: 1.3489 - accuracy: 0.5583
Epoch 9/38
4/4 [==============================] - 0s 39ms/step - loss: 1.2479 - accuracy: 0.6500
Epoch 10/38
4/4 [==============================] - 0s 35ms/step - loss: 1.1621 - accuracy: 0.6667
Epoch 11/38
4/4 [============

In [12]:
#Random Forest (RF) classifier
#Reshape
reshaped_labeled_set_view2 = labeled_set_view2.reshape(labeled_set_view2.shape[0], -1)
reshaped_label_labeled_view2 = label_labeled_set_view2.ravel()

#Random Forest
rf_classifier = RandomForestClassifier(n_estimators=20, random_state=42)
rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)

RandomForestClassifier(n_estimators=20, random_state=42)

In [ ]:
reshaped_unlabeled_set_view2 = unlabeled_set_view2.reshape(unlabeled_set_view2.shape[0], -1)

def co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2,  label_test_set_view1, label_test_set_view2 , threshold_confidence = 0.9):
    for trained in range(10):
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
        predictions1 = cnn_classifier.predict(unlabeled_set_view1)
        predictions2 = rf_classifier.predict(reshaped_unlabeled_set_view2)
        
        high_confidence_view1 = [instance for instance, prediction in zip(unlabeled_set_view1, predictions1) if np.greater_equal(prediction, threshold_confidence).any()]
        high_confidence_view2 = [instance for instance, prediction in zip(reshaped_unlabeled_set_view2, predictions2) if np.greater_equal(prediction, threshold_confidence).any()]
        
        labeled_set_view1 = labeled_set_view1.tolist()
        reshaped_labeled_set_view2 = reshaped_labeled_set_view2.tolist()

        labeled_set_view1.extend(high_confidence_view2)
        reshaped_labeled_set_view2.extend(high_confidence_view1)
        
        unlabeled_set_view1 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view1).any()])
        unlabeled_set_view2 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view2).any()])
        
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
    test_predictions1 = cnn_classifier.predict(label_test_set_view1)
    test_predictions2 = rf_classifier.predict(label_test_set_view2)
    
    combined_predictions = [prediction1 if prediction1 == prediction2 else None for prediction1, prediction2 in zip(test_predictions1, test_predictions2)]

    true_labels1 = label_labeled_set_view1
    true_labels2 = label_labeled_set_view2
    
    accuracy1 = accuracy_score(true_labels1, test_predictions1)
    accuracy2 = accuracy_score(true_labels2, test_predictions2)
    
    print(accuracy1)
    print(accuracy2)

    
    return trained

accuracy1, accuracy2 = co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2, label_test_set_view1, label_test_set_view2, threshold_confidence=0.9)
    
 


Epoch 1/38
4/4 [==============================] - 0s 39ms/step - loss: 0.5495 - accuracy: 0.8417
Epoch 2/38
4/4 [==============================] - 0s 40ms/step - loss: 0.4967 - accuracy: 0.8833
Epoch 3/38
4/4 [==============================] - 0s 51ms/step - loss: 0.5414 - accuracy: 0.8250
Epoch 4/38
4/4 [==============================] - 0s 48ms/step - loss: 0.4571 - accuracy: 0.8333
Epoch 5/38
4/4 [==============================] - 0s 43ms/step - loss: 0.4330 - accuracy: 0.8333
Epoch 6/38
4/4 [==============================] - 0s 40ms/step - loss: 0.4731 - accuracy: 0.8250
Epoch 7/38
4/4 [==============================] - 0s 53ms/step - loss: 0.4518 - accuracy: 0.8583
Epoch 8/38
4/4 [==============================] - 0s 37ms/step - loss: 0.4977 - accuracy: 0.8333
Epoch 9/38
4/4 [==============================] - 0s 41ms/step - loss: 0.4392 - accuracy: 0.8667
Epoch 10/38
4/4 [==============================] - 0s 36ms/step - loss: 0.4791 - accuracy: 0.8333
Epoch 11/38
4/4 [============

In [ ]:
#1-4. pick one of the classifiers and do the supervised training with the labeled data and calculate the accuracy

cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=10)




In [ ]:
Answer: ''' The co-training technique outperforms the convoluted neural network model when employed alone. 
Convoluted neural networks and random forest classifiers are used as models in our co-training model, with each operating on a separate view.
This is due to the iterative nature of co-training, which refines projected labels until a specific level of confidence is reached. 
The use of two classifiers and two perspectives provides more robust coverage. 
They also complement each other if one model has flaws. '''

In [ ]:
#Part 2. Label Propagation for Sea Ice Classification

In [ ]:
 #Apply the K-Nearest Neighbors (KNN) algorithm with a parameter configuration where n_neighbors is set to 7 for the label propagation model. Utilize one of the labeled data views and the corresponding unlabeled data from part 1 as input.

    
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_labeled = labeled_set_view1
y_labeled = labeled_set_view2
X_unlabeled = np.concatenate((unlabeled_set_view1, unlabeled_set_view2))

X = np.concatenate((X_labeled, X_unlabeled))
y = np.concatenate((y_labeled, [-1] * len(X_unlabeled))
                   
# Create label propagation model with KNN algorithm
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=7)

label_prop_model.fit(X, y)
                   
unlabeled_predictions = label_prop_model.transduction_[len(X_labeled):]
                   
semi_supervised_accuracy = accuracy_score(unlabeled_set_view2, unlabeled_predictions)
print(f'Semi-Supervised Accuracy: {semi_supervised_accuracy:.2f}')

                   

In [ ]:
#Part 3. Now let's perform some experimentation and make some observations

#threshold confidence= 0.8
reshaped_unlabeled_set_view2 = unlabeled_set_view2.reshape(unlabeled_set_view2.shape[0], -1)

def co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2,  label_test_set_view1, label_test_set_view2 , threshold_confidence = 0.8):
    for trained in range(10):
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
        predictions1 = cnn_classifier.predict(unlabeled_set_view1)
        predictions2 = rf_classifier.predict(reshaped_unlabeled_set_view2)
        
        high_confidence_view1 = [instance for instance, prediction in zip(unlabeled_set_view1, predictions1) if np.greater_equal(prediction, threshold_confidence).any()]
        high_confidence_view2 = [instance for instance, prediction in zip(reshaped_unlabeled_set_view2, predictions2) if np.greater_equal(prediction, threshold_confidence).any()]
        
        labeled_set_view1 = labeled_set_view1.tolist()
        reshaped_labeled_set_view2 = reshaped_labeled_set_view2.tolist()

        labeled_set_view1.extend(high_confidence_view2)
        reshaped_labeled_set_view2.extend(high_confidence_view1)
        
        unlabeled_set_view1 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view1).any()])
        unlabeled_set_view2 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view2).any()])
        
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
    test_predictions1 = cnn_classifier.predict(label_test_set_view1)
    test_predictions2 = rf_classifier.predict(label_test_set_view2)
    
    combined_predictions = [prediction1 if prediction1 == prediction2 else None for prediction1, prediction2 in zip(test_predictions1, test_predictions2)]

    true_labels1 = label_labeled_set_view1
    true_labels2 = label_labeled_set_view2
    
    accuracy1 = accuracy_score(true_labels1, test_predictions1)
    accuracy2 = accuracy_score(true_labels2, test_predictions2)
    
    print(accuracy1)
    print(accuracy2)

    
    return trained

accuracy1, accuracy2 = co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2, label_test_set_view1, label_test_set_view2, threshold_confidence=0.9)
    
 


In [ ]:
#Part 3. Now let's perform some experimentation and make some observations

#threshold confidence= 0.7
reshaped_unlabeled_set_view2 = unlabeled_set_view2.reshape(unlabeled_set_view2.shape[0], -1)

def co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2,  label_test_set_view1, label_test_set_view2 , threshold_confidence = 0.7):
    for trained in range(10):
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
        predictions1 = cnn_classifier.predict(unlabeled_set_view1)
        predictions2 = rf_classifier.predict(reshaped_unlabeled_set_view2)
        
        high_confidence_view1 = [instance for instance, prediction in zip(unlabeled_set_view1, predictions1) if np.greater_equal(prediction, threshold_confidence).any()]
        high_confidence_view2 = [instance for instance, prediction in zip(reshaped_unlabeled_set_view2, predictions2) if np.greater_equal(prediction, threshold_confidence).any()]
        
        labeled_set_view1 = labeled_set_view1.tolist()
        reshaped_labeled_set_view2 = reshaped_labeled_set_view2.tolist()

        labeled_set_view1.extend(high_confidence_view2)
        reshaped_labeled_set_view2.extend(high_confidence_view1)
        
        unlabeled_set_view1 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view1).any()])
        unlabeled_set_view2 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view2).any()])
        
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
    test_predictions1 = cnn_classifier.predict(label_test_set_view1)
    test_predictions2 = rf_classifier.predict(label_test_set_view2)
    
    combined_predictions = [prediction1 if prediction1 == prediction2 else None for prediction1, prediction2 in zip(test_predictions1, test_predictions2)]

    true_labels1 = label_labeled_set_view1
    true_labels2 = label_labeled_set_view2
    
    accuracy1 = accuracy_score(true_labels1, test_predictions1)
    accuracy2 = accuracy_score(true_labels2, test_predictions2)
    
    print(accuracy1)
    print(accuracy2)

    
    return trained

accuracy1, accuracy2 = co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2, label_test_set_view1, label_test_set_view2, threshold_confidence=0.9)
    
 


In [ ]:
#Part 3. Now let's perform some experimentation and make some observations

#threshold confidence= 0.6
reshaped_unlabeled_set_view2 = unlabeled_set_view2.reshape(unlabeled_set_view2.shape[0], -1)

def co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2,  label_test_set_view1, label_test_set_view2 , threshold_confidence = 0.6):
    for trained in range(10):
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
        predictions1 = cnn_classifier.predict(unlabeled_set_view1)
        predictions2 = rf_classifier.predict(reshaped_unlabeled_set_view2)
        
        high_confidence_view1 = [instance for instance, prediction in zip(unlabeled_set_view1, predictions1) if np.greater_equal(prediction, threshold_confidence).any()]
        high_confidence_view2 = [instance for instance, prediction in zip(reshaped_unlabeled_set_view2, predictions2) if np.greater_equal(prediction, threshold_confidence).any()]
        
        labeled_set_view1 = labeled_set_view1.tolist()
        reshaped_labeled_set_view2 = reshaped_labeled_set_view2.tolist()

        labeled_set_view1.extend(high_confidence_view2)
        reshaped_labeled_set_view2.extend(high_confidence_view1)
        
        unlabeled_set_view1 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view1).any()])
        unlabeled_set_view2 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view2).any()])
        
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
    test_predictions1 = cnn_classifier.predict(label_test_set_view1)
    test_predictions2 = rf_classifier.predict(label_test_set_view2)
    
    combined_predictions = [prediction1 if prediction1 == prediction2 else None for prediction1, prediction2 in zip(test_predictions1, test_predictions2)]

    true_labels1 = label_labeled_set_view1
    true_labels2 = label_labeled_set_view2
    
    accuracy1 = accuracy_score(true_labels1, test_predictions1)
    accuracy2 = accuracy_score(true_labels2, test_predictions2)
    
    print(accuracy1)
    print(accuracy2)

    
    return trained

accuracy1, accuracy2 = co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2, label_test_set_view1, label_test_set_view2, threshold_confidence=0.9)
    
 


In [ ]:
#Part 3 answer


In [ ]:
#3-2. Change the parameters of the K-Nearest Neighbors (KNN) algorithm for Label Propagation (part 2) with the values 3, 5, and 10, and explain what you understand about these parameter adjustments.

 #Apply the K-Nearest Neighbors (KNN) algorithm with a parameter configuration where n_neighbors is set to 3 for the label propagation model. Utilize one of the labeled data views and the corresponding unlabeled data from part 1 as input.

    
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_labeled = labeled_set_view1
y_labeled = labeled_set_view2
X_unlabeled = np.concatenate((unlabeled_set_view1, unlabeled_set_view2))

X = np.concatenate((X_labeled, X_unlabeled))
y = np.concatenate((y_labeled, [-1] * len(X_unlabeled))
                   
# Create label propagation model with KNN algorithm
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=3)

label_prop_model.fit(X, y)
                   
unlabeled_predictions = label_prop_model.transduction_[len(X_labeled):]
                   
semi_supervised_accuracy = accuracy_score(unlabeled_set_view2, unlabeled_predictions)
print(f'Semi-Supervised Accuracy: {semi_supervised_accuracy:.2f}')

                   

In [ ]:
 #Apply the K-Nearest Neighbors (KNN) algorithm with a parameter configuration where n_neighbors is set to 5 for the label propagation model. Utilize one of the labeled data views and the corresponding unlabeled data from part 1 as input.

    
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_labeled = labeled_set_view1
y_labeled = labeled_set_view2
X_unlabeled = np.concatenate((unlabeled_set_view1, unlabeled_set_view2))

X = np.concatenate((X_labeled, X_unlabeled))
y = np.concatenate((y_labeled, [-1] * len(X_unlabeled))
                   
# Create label propagation model with KNN algorithm
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=5)

label_prop_model.fit(X, y)
                   
unlabeled_predictions = label_prop_model.transduction_[len(X_labeled):]
                   
semi_supervised_accuracy = accuracy_score(unlabeled_set_view2, unlabeled_predictions)
print(f'Semi-Supervised Accuracy: {semi_supervised_accuracy:.2f}')

                   

In [ ]:
 #Apply the K-Nearest Neighbors (KNN) algorithm with a parameter configuration where n_neighbors is set to 10 for the label propagation model. Utilize one of the labeled data views and the corresponding unlabeled data from part 1 as input.

    
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_labeled = labeled_set_view1
y_labeled = labeled_set_view2
X_unlabeled = np.concatenate((unlabeled_set_view1, unlabeled_set_view2))

X = np.concatenate((X_labeled, X_unlabeled))
y = np.concatenate((y_labeled, [-1] * len(X_unlabeled))
                   
# Create label propagation model with KNN algorithm
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=7)

label_prop_model.fit(X, y)
                   
unlabeled_predictions = label_prop_model.transduction_[len(X_labeled):]
                   
semi_supervised_accuracy = accuracy_score(unlabeled_set_view2, unlabeled_predictions)
print(f'Semi-Supervised Accuracy: {semi_supervised_accuracy:.2f}')

                   

In [ ]:
# Sequential model
cnn_classifier = Sequential()

#1. Input Layer: BatchNormalization with input shape (32, 32, 1).
cnn_classifier.add(BatchNormalization(input_shape=(32, 32, 1)))

#2. Convolutional Layer 1: 32 filters, each with a 3x3 kernel and ReLU activation.
cnn_classifier.add(Conv2D(32, (3, 3), activation='relu'))

#3. Max Pooling Layer 1: 2x2 pooling with a stride of 2.
cnn_classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))


#8. BatchNormalization layer
cnn_classifier.add(BatchNormalization())

#9. Flatten
cnn_classifier.add(Flatten())

#10. Dropout layer-Dropout=0.1
cnn_classifier.add(Dropout(0.1))

#11. Fully Connected Layer 1: 16 neurons, ReLU activation, and L2 regularization with a weight decay of 0.001.
cnn_classifier.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)))

#12. Dropout Layer with a dropout rate of 0.1.
cnn_classifier.add(Dropout(0.1))

#13. Fully Connected Layer 2: 16 neurons, ReLU activation, and L2 regularization with a weight decay of 0.001.
cnn_classifier.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)))

#14. Dropout layer-Dropout=0.1
cnn_classifier.add(Dropout(0.1))

#15. Output Layer: Dense layer with the number of neurons equal to the number of classes and softmax activation.
cnn_classifier.add(Dense(6, activation='softmax'))

cnn_classifier.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)

In [ ]:
#Random Forest (RF) classifier
#Reshape
reshaped_labeled_set_view2 = labeled_set_view2.reshape(labeled_set_view2.shape[0], -1)
reshaped_label_labeled_view2 = label_labeled_set_view2.ravel()

#Random Forest
rf_classifier = RandomForestClassifier(n_estimators=1, random_state=42)
rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)

In [ ]:
reshaped_unlabeled_set_view2 = unlabeled_set_view2.reshape(unlabeled_set_view2.shape[0], -1)

def co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2,  label_test_set_view1, label_test_set_view2 , threshold_confidence = 0.9):
    for trained in range(10):
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
        predictions1 = cnn_classifier.predict(unlabeled_set_view1)
        predictions2 = rf_classifier.predict(reshaped_unlabeled_set_view2)
        
        high_confidence_view1 = [instance for instance, prediction in zip(unlabeled_set_view1, predictions1) if np.greater_equal(prediction, threshold_confidence).any()]
        high_confidence_view2 = [instance for instance, prediction in zip(reshaped_unlabeled_set_view2, predictions2) if np.greater_equal(prediction, threshold_confidence).any()]
        
        labeled_set_view1 = labeled_set_view1.tolist()
        reshaped_labeled_set_view2 = reshaped_labeled_set_view2.tolist()

        labeled_set_view1.extend(high_confidence_view2)
        reshaped_labeled_set_view2.extend(high_confidence_view1)
        
        unlabeled_set_view1 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view1).any()])
        unlabeled_set_view2 = np.array([instance for instance in reshaped_unlabeled_set_view2 if not np.isin(instance, high_confidence_view2).any()])
        
        cnn_classifier.fit(labeled_set_view1, label_labeled_set_view1, epochs=38)
        rf_classifier.fit(reshaped_labeled_set_view2, reshaped_label_labeled_view2)
        
    test_predictions1 = cnn_classifier.predict(label_test_set_view1)
    test_predictions2 = rf_classifier.predict(label_test_set_view2)
    
    combined_predictions = [prediction1 if prediction1 == prediction2 else None for prediction1, prediction2 in zip(test_predictions1, test_predictions2)]

    true_labels1 = label_labeled_set_view1
    true_labels2 = label_labeled_set_view2
    
    accuracy1 = accuracy_score(true_labels1, test_predictions1)
    accuracy2 = accuracy_score(true_labels2, test_predictions2)
    
    print(accuracy1)
    print(accuracy2)

    
    return trained

accuracy1, accuracy2 = co_training(cnn_classifier, rf_classifier, labeled_set_view1, reshaped_labeled_set_view2, unlabeled_set_view1, reshaped_unlabeled_set_view2, label_test_set_view1, label_test_set_view2, threshold_confidence=0.9)
    
 

In [ ]:

cnn_classifier.fit(labeled_set_II, label_labeled_set_II, epochs=10)
predictions = cnn_classifier.predict(test_set)
accuracy = accuracy_score(test_labels, predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")
